In [1]:
!pip install matplotlib tensorflow_model_optimization tf_keras image_classifiers
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import cifar100
import tensorflow_model_optimization as tfmot
from tqdm import tqdm
import numpy as np
import tensorflow as tf
import tf_keras as keras
from tf_keras.applications import ResNet50V2
from tf_keras.datasets import cifar100
from tf_keras import Sequential, Input
from tf_keras.layers import Dense, Dropout, RandomFlip, RandomTranslation, RandomRotation,RandomBrightness, RandomContrast, RandomZoom, GlobalAveragePooling2D
from tf_keras.applications.resnet_v2 import preprocess_input
from tf_keras.models import Model
from tf_keras.activations import linear
from classification_models.models_factory import ModelsFactory
import tempfile
import os
import zipfile

2024-05-05 19:55:35.302732: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-05 19:55:35.375360: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-05 19:55:36.500461: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
class KerasModelsFactory(ModelsFactory):
    @staticmethod
    def get_kwargs():
        return {
            'backend': keras.backend,
            'layers': keras.layers,
            'models': keras.models,
            'utils': keras.utils,
        }
# Pour faire marcher image-classifiers avec tf_keras, la version compatibilité de tf.keras, nécéssaire pour tflite
Classifiers = KerasModelsFactory()

In [3]:
n_epoch = 10
batch_size = 100
num_classes = 100
n_images = 50000 # Pour l'entrainement, et 10000 pour le test

In [4]:
!mc cp s3/afeldmann/projet_cnam/modele_enseignant.keras /home/onyxia/work/projet_distillation_cnam/sauvegardes/modele_enseignant.keras
model_enseignant = Sequential([
    Input((224,224,3)),
    ResNet50V2(include_top=False, weights='imagenet', pooling="avg"),
    Dropout(0.25),
    Dense(256, activation="sigmoid", kernel_regularizer = keras.regularizers.L1(0.001)),
    Dropout(0.5),
    Dense(num_classes, activation="softmax", kernel_regularizer = keras.regularizers.L2(0.001))
])
# Keras 3.1.1 est buggé et le chargement direct ne marche pas ici, même si les poids sont bien enregistrés
model_enseignant.load_weights("/home/onyxia/work/projet_distillation_cnam/sauvegardes/modele_enseignant.keras")

model_enseignant.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

...nant.keras: 135.73 MiB / 135.73 MiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 110.58 MiB/s 1s

2024-05-05 19:55:44.411872: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13290 MB memory:  -> device: 0, name: NVIDIA A2, pci bus id: 0000:ca:00.0, compute capability: 8.6


In [5]:
def get_modele_logits(modele):
    config = modele.layers[-1].get_config()
    config['activation'] = linear
    config['name'] = 'logits'
    res = Model(inputs=modele.inputs, outputs=[Dense(**config)(modele.layers[-2].output)])
    res.layers[-1].set_weights([x.numpy() for x in modele.layers[-1].weights])
    res.compile(metrics=['accuracy'])
    return res

model_enseignant_logits = get_modele_logits(model_enseignant)

In [6]:
def ResNet18():
    resnet18, preprocess_input = Classifiers.get('resnet18')
    resnet = resnet18((224, 224, 3), weights='imagenet', include_top=False)
    resnet_output = GlobalAveragePooling2D()(resnet.output)
    resnet = Model(inputs=resnet.input, outputs=resnet_output)
    return resnet

def new_modele_resnet():
    model = Sequential([
        Input((224,224,3)),
        ResNet18(),
        Dropout(0.25),
        Dense(256, activation="sigmoid", kernel_regularizer = keras.regularizers.L1(0.001)),
        Dropout(0.5),
        Dense(num_classes, activation="softmax", kernel_regularizer = keras.regularizers.L2(0.001))
    ])
    model.compile(metrics=['accuracy'])
    return model

In [7]:
!mc cp s3/afeldmann/projet_cnam/model_etudiant_t3_a50.keras /home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant.keras
model_etudiant = new_modele_resnet()
model_etudiant.load_weights("/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant.keras")
model_etudiant.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

..._a50.keras: 43.54 MiB / 43.54 MiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 107.32 MiB/s 0s

In [8]:
@tf.function
def softmax(logits, temp):
    expo = tf.exp(logits / temp)
    return expo / tf.reduce_sum(expo, axis = 1, keepdims=True)

def preprocessing(image, label):
    image = tf.image.resize(image, (224, 224))
    label = tf.squeeze(tf.one_hot(label, depth = num_classes), axis = 0)
    return  image, label

augmentation_donnees_keras = Sequential([
    RandomFlip("horizontal"),
    RandomTranslation(0.2,0.2),
    RandomRotation(0.2),
    RandomZoom(0.2),
    RandomContrast(0.2),
    RandomBrightness(0.2,value_range=(0,1))
])

def augmentation_donnees(image, label):
    return augmentation_donnees_keras(image/255.0, training = True)*255.0, label

def preprocess_resnet(image, label):
    return preprocess_input(image), label

def mix_labels(image,label):
    logits = model_enseignant_logits(image, training = False)
    label = (softmax(logits,3) + label) / 2
    return image, label

train_dataset, test_dataset = cifar100.load_data()
train_dataset = (
    tf.data.Dataset.from_tensor_slices(train_dataset)
    .map(preprocessing)
    .cache()
    .shuffle(n_images)
    .batch(batch_size)
    .map(augmentation_donnees, num_parallel_calls = tf.data.AUTOTUNE)
    .map(preprocess_resnet, num_parallel_calls = tf.data.AUTOTUNE)
    .map(mix_labels, num_parallel_calls = tf.data.AUTOTUNE)
    .prefetch(tf.data.AUTOTUNE)
)

test_dataset = (
    tf.data.Dataset.from_tensor_slices(test_dataset)
    .map(preprocessing)
    .batch(batch_size)
    .map(preprocess_resnet, num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(tf.data.AUTOTUNE)
)

In [9]:
end_step = np.ceil(n_images / batch_size).astype(np.int32) * n_epoch

pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.20,
                                                               final_sparsity=0.60,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_etudiant_pruning = tfmot.sparsity.keras.prune_low_magnitude(model_etudiant, **pruning_params)

In [10]:
callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep()
]

model_etudiant_pruning.compile(optimizer=keras.optimizers.SGD(learning_rate=0.005), loss='categorical_crossentropy', metrics=['accuracy'])

model_etudiant_pruning.fit(train_dataset, callbacks=callbacks, epochs=n_epoch)

Epoch 1/10


2024-05-05 19:57:26.994233: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8900
I0000 00:00:1714939077.998705   90877 service.cc:145] XLA service 0x7f4a003c2ec0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1714939077.998842   90877 service.cc:153]   StreamExecutor device (0): NVIDIA A2, Compute Capability 8.6
I0000 00:00:1714939084.482713   90877 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


500/500 [==============================] - 470s 640ms/step - loss: 10.5920 - accuracy: 0.6820
Epoch 2/10
500/500 [==============================] - 407s 785ms/step - loss: 9.8944 - accuracy: 0.7152
Epoch 3/10
500/500 [==============================] - 411s 810ms/step - loss: 9.3136 - accuracy: 0.7258
Epoch 4/10
500/500 [==============================] - 403s 773ms/step - loss: 8.8127 - accuracy: 0.7264
Epoch 5/10
500/500 [==============================] - 400s 767ms/step - loss: 8.3976 - accuracy: 0.7272
Epoch 6/10
500/500 [==============================] - 402s 772ms/step - loss: 8.0605 - accuracy: 0.7254
Epoch 7/10
500/500 [==============================] - 406s 774ms/step - loss: 7.7973 - accuracy: 0.7259
Epoch 8/10
500/500 [==============================] - 387s 740ms/step - loss: 7.5884 - accuracy: 0.7229
Epoch 9/10
500/500 [==============================] - 403s 774ms/step - loss: 7.4205 - accuracy: 0.7224
Epoch 10/10
500/500 [==============================] - 386s 744ms/step - l

In [11]:
nom_modele =  f"model_etudiant_pruning"
wd = os.getcwd()
model_etudiant_pruning.save(f"{wd}/sauvegardes/{nom_modele}.keras")
os.system(f"mc cp {wd}/sauvegardes/{nom_modele}.keras s3/afeldmann/projet_cnam/{nom_modele}.keras")
os.system(f"mc cp {wd}/sauvegardes/{nom_modele}_logs.csv s3/afeldmann/projet_cnam/{nom_modele}_logs.csv")

`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_pruning.keras` -> `s3/afeldmann/projet_cnam/model_etudiant_pruning.keras`
Total: 173.50 MiB, Transferred: 173.50 MiB, Speed: 220.74 MiB/s
`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_pruning_logs.csv` -> `s3/afeldmann/projet_cnam/model_etudiant_pruning_logs.csv`
Total: 154 B, Transferred: 154 B, Speed: 1.72 KiB/s


0

In [12]:
model_etudiant_pruning.compile(metrics="accuracy")
model_etudiant_pruning.evaluate(test_dataset)
model_etudiant_pruning.evaluate(test_dataset.map(lambda images, labels: (images,model_enseignant(images, training = False)), num_parallel_calls = tf.data.AUTOTUNE))

100/100 [==============================] - 67s 594ms/step - loss: 3.9977 - accuracy: 0.7744


[3.9977033138275146, 0.774399995803833]

In [23]:
pruned_keras_file = "/home/onyxia/work/projet_distillation_cnam/sauvegardes/modele_etudiant_pruning.h5"
model_for_export = tfmot.sparsity.keras.strip_pruning(model_etudiant_pruning)
keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)

/tmp/ipykernel_90549/2267512799.py:3: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)


In [25]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
pruned_tflite_model = converter.convert()
pruned_tflite_file = "/home/onyxia/work/projet_distillation_cnam/sauvegardes/modele_etudiant_pruning.tflite"
with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)


INFO:tensorflow:Assets written to: /tmp/tmp7ds_4183/assets


INFO:tensorflow:Assets written to: /tmp/tmp7ds_4183/assets
W0000 00:00:1714944617.616395   90549 tf_tfl_flatbuffer_helpers.cc:390] Ignored output_format.
W0000 00:00:1714944617.616442   90549 tf_tfl_flatbuffer_helpers.cc:393] Ignored drop_control_dependency.
2024-05-05 21:30:17.616738: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp7ds_4183
2024-05-05 21:30:17.630934: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-05-05 21:30:17.630964: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp7ds_4183
2024-05-05 21:30:17.725272: I tensorflow/cc/saved_model/loader.cc:234] Restoring SavedModel bundle.
2024-05-05 21:30:17.986315: I tensorflow/cc/saved_model/loader.cc:218] Running initialization op on SavedModel bundle at path: /tmp/tmp7ds_4183
2024-05-05 21:30:18.065788: I tensorflow/cc/saved_model/loader.cc:317] SavedModel load for tags { serve }; Status: success: OK. Took 449054

In [26]:
test_batches = (
    test_dataset
    .map(lambda images, labels: (images,labels,model_enseignant(images, training = False)), num_parallel_calls = tf.data.AUTOTUNE)
    .unbatch()
    .batch(1)
)

interpreter = tf.lite.Interpreter(model_path=pruned_tflite_file)
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

test_etupreds, test_labels, test_enspreds = [], [], []
for img, label, enspred in tqdm(test_batches.take(1000)):
    interpreter.set_tensor(input_index, img)
    interpreter.invoke()
    test_etupreds.append(interpreter.get_tensor(output_index))
    test_labels.append(label.numpy()[0])
    test_enspreds.append(enspred.numpy()[0])

score_dur, score_dis = 0, 0
for item in range(0,len(test_etupreds)):
    etupred = np.argmax(test_etupreds[item])
    label = np.argmax(test_labels[item])
    enspred = np.argmax(test_enspreds[item])
    if etupred == label:
        score_dur += 1
    if etupred == enspred:
        score_dis += 1

print(f"{score_dur} prédictions correctes sur 1000 face aux étiquettes dures et {score_dis} face aux étiquettes douces.")

999it [02:26, 15.28it/s]2024-05-05 21:32:55.252805: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
1000it [02:26,  6.82it/s]

764 prédictions correctes sur 1000 face aux étiquettes dures et 770 face aux étiquettes douces.


In [27]:
def get_gzipped_model_size(file):
  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)
  return os.path.getsize(zipped_file)

In [28]:
print("Taille du modèle enseignant zippé : %.2f bytes" % (get_gzipped_model_size("/home/onyxia/work/projet_distillation_cnam/sauvegardes/modele_enseignant.keras")))
print("Taille du modèle étudiant zippé : %.2f bytes" % (get_gzipped_model_size("/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant.keras")))
print("Taille du modèle étudiant zippé après élagage : %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
print("Taille du modèle étudiant zippé après élagage et quantification : %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))

Taille du modèle enseignant zippé : 131559019.00 bytes
Taille du modèle étudiant zippé : 42144643.00 bytes
Taille du modèle étudiant zippé après élagage : 21795854.00 bytes
Taille du modèle étudiant zippé après élagage et quantification : 6221354.00 bytes


In [29]:
!mc cp /home/onyxia/work/projet_distillation_cnam/sauvegardes/modele_etudiant_pruning.tflite s3/afeldmann/projet_cnam/modele_etudiant_pruning.tflite
!mc cp /home/onyxia/work/projet_distillation_cnam/sauvegardes/modele_etudiant_pruning.h5 s3/afeldmann/projet_cnam/modele_etudiant_pruning.h5

...pruning.h5: 43.42 MiB / 43.42 MiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 99.89 MiB/s 0s]11;?\